In [3]:
import pandas as pd
from Bio import SeqIO
from Bio.ExPASy import ScanProsite

So I dun goofed and forgot to have you you guys download sequences as well for this next step, below is a script you help you merge the two files automatically, you'll use a similar pattern for the next set of steps which invovles extracting protien domains. 

In [4]:
#Read your blast results csv in
df = pd.read_csv("C:\\Users\\cmgro\\Downloads\\hdeD_genome_feature (9).csv")
#Show the first 5 rows
print(df.head())
#Temp lists to attatch variables 
ids = []
seqs = []
#Seq.IO.parse is. an easy way to read through every gene/prot in a fasta file 
for record in SeqIO.parse("C:\\Users\\cmgro\\Downloads\\hdeD_genome_feature (9).fasta", "fasta"):
    # So the fasta ID line is fig|1240239.3.peg.2328|XFF4834R_chr22840|VBIXanFus78442_2328|
    # but we only need the first two elements ("fig and 1240239.3.peg.2328") to make fig|1240239.3.peg.2328
    # We then use this ID to merge back onto the original dataframe (basically we say if this row has this ID, add the sequence to that row)
    
    #Fancy way of extracting the ID
    ids.append("|".join(record.id.split("|")[0:2]))
    #Fancy way of extracting the sequence 
    seqs.append(str(record.seq))
#making a new dataframe from this info 
seq_df = pd.DataFrame.from_dict({"PATRIC ID": ids, "Sequences":seqs})
print(seq_df.head())
#Mergine back and saving 
df = df.merge(seq_df, on="PATRIC ID")
df.to_csv("/Users/cmgro/Downloads/merged_hdeD.csv")

                              Genome  Genome ID     Accession  \
0  Yokenella regensburgei ATCC 43003  1002368.3  AGCL01000039   
1    Bradyrhizobium japonicum USDA 6  1037409.3      AP012206   
2     Bradyrhizobium elkanii WSM1741  1038859.3  AXAU01000001   
3         Bradyrhizobium sp. WSM3983  1038867.3  AXAY01000006   
4          Mesorhizobium sp. WSM4349  1040988.3  ARDH01000011   

                PATRIC ID  RefSeq Locus Tag         Alt Locus Tag  \
0  fig|1002368.3.peg.2679  HMPREF0880_02940  VBIYokReg189340_2679   
1  fig|1037409.3.peg.5438        BJ6T_54170  VBIBraJap202769_5438   
2   fig|1038859.3.peg.922               NaN  VBIBraElk247588_0922   
3  fig|1038867.3.peg.2509               NaN   VBIBraSp248367_2509   
4  fig|1040988.3.peg.4335               NaN   VBIMesSp242480_4335   

                                          Feature ID Annotation Feature Type  \
0  PATRIC.1002368.3.AGCL01000039.CDS.217161.21773...     PATRIC          CDS   
1  PATRIC.1037409.3.AP012206.CDS.5

In [ ]:
import time
sigs = []

for i in df["Sequences"].values:
    handle = ScanProsite.scan(seq=i, mirror="https://prosite.expasy.org")
    result = ScanProsite.read(handle)
    qq = []
    for i in result:
        print(i)
        qq.append(i["signature_ac"])
    time.sleep(5)
    qq = set(qq)
    sigs.append(qq)
    
df["Prosite results"] = sigs
df.to_csv("/Users/christine/Downloads/hdeD_motifs.csv")

{'sequence_ac': 'USERSEQ1', 'start': 128, 'stop': 139, 'signature_ac': 'PS00213', 'level_tag': '(-1)'}


In [ ]:
df
